<a href="https://colab.research.google.com/github/eherrador/PredictingLongShortTermMemory/blob/main/PredictingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1: Import Libraries

In [4]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

### Step 2: Load and Prepare the Dataset

In [ ]:
# Load your dataset
data = pd.read_csv('Melate.csv', header=None)

# Show the first row of DataFrame
print(data.head())

# Convert data to numpy array
data = data.values

# Normalize the dataset
scaler = MinMaxScaler(feature_range=(1, 56))  # Normalize between 1 and 56
data_scaled = scaler.fit_transform(data)

    0   1   2   3   4   5   6
0  10  17  18  32  35  42  27
1   6  15  16  22  39  56   9
2  15  35  50  51  52  54  28
3   4  14  15  30  31  56  23
4   8  32  42  45  48  50   7


### Step 3: Create Sequences for LSTM

In [ ]:
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), :])
        y.append(data[i + time_step, :])
    return np.array(X), np.array(y)

# Create sequences with a time step (number of previous rows to consider)
time_step = 10  # You can adjust this value
X, y = create_dataset(data_scaled, time_step)

### Step 4: Reshape Input for LSTM

In [ ]:
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])  # (samples, time steps, features)

### Step 5: Build the LSTM Model

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(7, activation='linear'))  # 7 outputs for the 7 integers

model.compile(optimizer='adam', loss='mean_squared_error')

### Step 6: Train the Model

In [ ]:
# model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))
model.fit(X, y, epochs=100, batch_size=32)

### Step 7: Make Predictions

In [ ]:
# Get the last 'time_step' rows
last_sequence = data_scaled[-time_step:]
last_sequence = last_sequence.reshape((1, last_sequence.shape[0], last_sequence.shape[1]))

# Make a prediction
predicted = model.predict(last_sequence)
predicted_row = scaler.inverse_transform(predicted)  # Rescale back to original range
print("Predicted next row:", predicted_row)